In [1]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings

warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [2]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по КРС.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,КРС
614,ГАЛМАТЫ,2016-03-01,94.38
584,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2023-09-01,4743.50
1830,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2023-03-01,3157.46
1002,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2021-12-01,18239.40
239,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-12-01,10603.99
1474,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2021-04-01,1663.80
735,ГАСТАНА,2016-04-01,9.89
37,АКМОЛИНСКАЯ ОБЛАСТЬ,2018-02-01,4100.47
365,АТЫРАУСКАЯ ОБЛАСТЬ,2015-06-01,1946.39
116,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-09-01,2909.66


In [8]:
regions = df['Регион'].unique()
target   = "КРС"
horizon  = 3
epsilon = 1e-6

In [9]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

In [ ]:
# results_hw = []
# for region in regions:
#     ts = (df[df["Регион"] == region]
#           .set_index("Период")[target]
#           .dropna()
#           .sort_index())
#     if len(ts) < 24:
#         print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
#         continue

#     for test_start in test_starts:
#         test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

#         # формируем train / test
#         train = ts[ts.index < test_start]
#         test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

#         # пропускаем, если недостаточно данных или неполный test
#         if len(train) < 24 or len(test) < horizon:
#             continue

#         # обучаем модель
#         model = ExponentialSmoothing(
#             train,
#             seasonal="add",
#             seasonal_periods=12
#         ).fit(optimized=True)

#         # прогноз и метрики
#         fc   = model.forecast(horizon)
#         rmse = np.sqrt(mean_squared_error(test, fc))
#         mae  = mean_absolute_error(test, fc)
#         mape = (np.abs((test - fc) / test).mean()) * 100

#         results_hw.append({
#             "Регион":      region,
#             "Test start":  test_start.strftime("%Y-%m"),
#             "Test end":    test_end.strftime("%Y-%m"),
#             "Forecast":    [x.round(2) for x in list(fc.values)],
#             "Actual":      [y.round(2) for y in list(test.values)],
#             "RMSE":        rmse,
#             "MAE":         mae,
#             "MAPE_%":      mape
#         })

# # 4) Усреднение по всем скользящим окнам для каждого региона
# res_hw = pd.DataFrame(results_hw)
# print("Результаты пронозов на 3 месяца:")

# display(res_hw)

# final_hw = (
#     res_hw
#     .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
#     .mean()
#     .round(2)
#     .reset_index()
# )
# print("Средние метрики Holt–Winters по регионам (rolling-3):")
# display(final_hw)

Результаты пронозов на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[4133.83, 3442.91, 4009.72]","[4035.37, 3446.55, 3779.21]",144.733915,110.870577,2.881671
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[3408.39, 3974.56, 3617.58]","[3446.55, 3779.21, 3691.05]",122.493482,102.325695,2.755561
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[3952.57, 3606.75, 3632.84]","[3779.21, 3691.05, 3574.86]",116.218621,105.211995,2.830961
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[3562.33, 3561.29, 3897.55]","[3691.05, 3574.86, 3951.41]",80.939247,65.383912,1.743354
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[3623.22, 3971.56, 1686.98]","[3574.86, 3951.41, 2029.47]",200.039163,136.999626,6.246213
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[7927.08, 7395.72, 9299.72]","[15715.54, 6948.86, 9372.79]",4504.264280,2769.464184,18.923096
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[8947.35, 10838.01, 11632.38]","[6948.86, 9372.79, 15738.07]",2768.723557,2523.132693,23.493433
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[9876.09, 10689.04, 11750.0]","[9372.79, 15738.07, 10566.5]",3008.138579,2245.277471,16.217317
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[10852.45, 11588.44, 11624.79]","[15738.07, 10566.5, 11441.98]",2883.693646,2030.122042,14.104168


Средние метрики Holt–Winters по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,235.27,200.03,7.62
1,АКТЮБИНСКАЯ ОБЛАСТЬ,964.89,842.97,15.52
2,АЛМАТИНСКАЯ ОБЛАСТЬ,1138.56,976.08,21.12
3,АТЫРАУСКАЯ ОБЛАСТЬ,229.38,204.48,13.45
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1891.76,1844.47,63.70
5,ГАЛМАТЫ,13.44,11.55,694.06
6,ГАСТАНА,1.08,0.92,35.79
7,ГШЫМКЕНТ,416.06,371.04,83.19
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,507.70,480.06,13.59
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,294.43,246.51,5.86


## Holw-Winter's (log)

In [11]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[4059.46, 3449.58, 3899.33]","[4035.37, 3446.55, 3779.21]",70.755483,49.081411,1.287815
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[3441.1, 3889.78, 3600.63]","[3446.55, 3779.21, 3691.05]",82.523185,68.810703,1.844455
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[3892.82, 3603.56, 3628.19]","[3779.21, 3691.05, 3574.86]",88.326413,84.808585,2.289403
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[3558.51, 3583.37, 3877.67]","[3691.05, 3574.86, 3951.41]",87.705400,71.597100,1.898370
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[3638.69, 3937.14, 1900.76]","[3574.86, 3951.41, 2029.47]",83.352679,68.934890,2.829502
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[7842.97, 7289.94, 8800.3]","[15715.54, 6948.86, 9372.79]",4561.484633,2928.713189,20.370201
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[7926.49, 9556.52, 10605.77]","[6948.86, 9372.79, 15738.07]",3018.278853,2097.886801,16.213301
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[9381.81, 10415.31, 11429.82]","[9372.79, 15738.07, 10566.5]",3113.262835,2065.033537,14.029166
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[10415.3, 11429.26, 11406.99]","[15738.07, 10566.5, 11441.98]",3113.275566,2073.507419,14.097286


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,153.57,129.92,4.77
1,АКТЮБИНСКАЯ ОБЛАСТЬ,778.20,600.08,10.54
2,АЛМАТИНСКАЯ ОБЛАСТЬ,1064.41,880.87,12.93
3,АТЫРАУСКАЯ ОБЛАСТЬ,169.73,148.49,8.87
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,414.51,357.77,9.46
5,ГАЛМАТЫ,3.84,2.82,100.69
6,ГАСТАНА,0.46,0.40,15.60
7,ГШЫМКЕНТ,232.30,203.83,43.10
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,497.74,430.01,10.74
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,260.95,218.99,5.22


## SARIMA

In [12]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(0, 1, 2)","(0, 1, 1, 12)",98.14,78.86,2.09,"[3893.47, 3352.97, 3778.1]","[4035.37, 3446.55, 3779.21]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(0, 1, 2)","(0, 1, 1, 12)",86.67,72.96,1.96,"[3454.4, 3894.89, 3595.7]","[3446.55, 3779.21, 3691.05]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(0, 1, 2)","(0, 1, 1, 12)",85.89,75.91,2.04,"[3890.09, 3593.82, 3594.47]","[3779.21, 3691.05, 3574.86]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(0, 1, 2)","(2, 1, 0, 12)",188.55,186.77,4.98,"[3496.1, 3423.01, 3737.89]","[3691.05, 3574.86, 3951.41]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(0, 1, 2)","(0, 1, 1, 12)",87.86,79.10,3.07,"[3656.87, 3920.52, 1905.06]","[3574.86, 3951.41, 2029.47]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(0, 1, 2)","(0, 1, 1, 12)",5706.81,3686.53,25.91,"[5868.41, 6375.49, 10011.88]","[15715.54, 6948.86, 9372.79]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(1, 0, 0)","(0, 1, 1, 12)",2955.49,2748.27,25.56,"[8881.75, 11399.94, 11453.29]","[6948.86, 9372.79, 15738.07]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(0, 0, 0)","(0, 1, 1, 12)",2735.27,1989.05,14.60,"[10634.39, 11173.69, 10707.65]","[9372.79, 15738.07, 10566.5]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(0, 0, 0)","(0, 1, 1, 12)",2702.99,1798.24,12.09,"[11086.6, 10883.37, 11015.59]","[15738.07, 10566.5, 11441.98]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,170.89,146.60,5.22
1,АКТЮБИНСКАЯ ОБЛАСТЬ,887.97,728.64,12.77
2,АЛМАТИНСКАЯ ОБЛАСТЬ,918.74,730.59,11.63
3,АТЫРАУСКАЯ ОБЛАСТЬ,160.94,142.53,8.61
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,439.20,383.63,10.50
5,ГАЛМАТЫ,4.46,3.69,87.36
6,ГАСТАНА,0.79,0.71,26.88
7,ГШЫМКЕНТ,204.56,135.17,28.85
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,415.41,360.33,9.24
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,238.56,206.83,4.57


## Facebook Prophet

In [14]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})

        m = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
        m.fit(df_prophet)

        # Создаем DataFrame будущих дат и делаем прогноз
        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # Отбираем только наши горизонты
        fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


10:23:21 - cmdstanpy - INFO - Chain [1] start processing
10:23:21 - cmdstanpy - INFO - Chain [1] done processing
10:23:22 - cmdstanpy - INFO - Chain [1] start processing
10:23:22 - cmdstanpy - INFO - Chain [1] done processing
10:23:22 - cmdstanpy - INFO - Chain [1] start processing
10:23:22 - cmdstanpy - INFO - Chain [1] done processing
10:23:22 - cmdstanpy - INFO - Chain [1] start processing
10:23:22 - cmdstanpy - INFO - Chain [1] done processing
10:23:23 - cmdstanpy - INFO - Chain [1] start processing
10:23:23 - cmdstanpy - INFO - Chain [1] done processing
10:23:23 - cmdstanpy - INFO - Chain [1] start processing
10:23:23 - cmdstanpy - INFO - Chain [1] done processing
10:23:23 - cmdstanpy - INFO - Chain [1] start processing
10:23:23 - cmdstanpy - INFO - Chain [1] done processing
10:23:23 - cmdstanpy - INFO - Chain [1] start processing
10:23:24 - cmdstanpy - INFO - Chain [1] done processing
10:23:24 - cmdstanpy - INFO - Chain [1] start processing
10:23:24 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,204.96,189.31,4.98,"[4232.15, 3536.13, 4060.78]","[4035.37, 3446.55, 3779.21]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,152.91,130.45,3.52,"[3498.45, 4019.08, 3591.47]","[3446.55, 3779.21, 3691.05]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,147.82,116.43,3.11,"[4012.2, 3585.42, 3585.52]","[3779.21, 3691.05, 3574.86]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,71.70,45.87,1.24,"[3567.27, 3566.16, 3946.29]","[3691.05, 3574.86, 3951.41]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,249.52,145.92,7.15,"[3570.18, 3952.35, 1597.32]","[3574.86, 3951.41, 2029.47]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,4135.42,2661.99,18.40,"[8586.5, 7137.9, 8704.9]","[15715.54, 6948.86, 9372.79]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,3437.12,2217.66,16.04,"[7659.53, 9341.09, 9827.45]","[6948.86, 9372.79, 15738.07]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,3580.13,2603.84,18.53,"[9357.79, 9832.47, 12457.42]","[9372.79, 15738.07, 10566.5]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,3681.23,3104.93,22.91,"[9845.21, 12469.43, 12960.98]","[15738.07, 10566.5, 11441.98]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,348.56,308.18,11.38
1,АКТЮБИНСКАЯ ОБЛАСТЬ,867.92,746.80,15.30
2,АЛМАТИНСКАЯ ОБЛАСТЬ,4097.33,3743.97,95.35
3,АТЫРАУСКАЯ ОБЛАСТЬ,305.92,269.17,18.16
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,3742.48,3410.64,111.41
5,ГАЛМАТЫ,13.42,11.62,795.82
6,ГАСТАНА,0.92,0.76,32.24
7,ГШЫМКЕНТ,416.82,344.15,69.97
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,690.90,566.08,13.11
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,234.01,201.49,4.83


In [20]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})

        m = Prophet()
        m.fit(df_prophet)

        # Создаем DataFrame будущих дат и делаем прогноз
        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # Отбираем только наши горизонты
        fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


12:45:33 - cmdstanpy - INFO - Chain [1] start processing
12:45:34 - cmdstanpy - INFO - Chain [1] done processing
12:45:34 - cmdstanpy - INFO - Chain [1] start processing
12:45:34 - cmdstanpy - INFO - Chain [1] done processing
12:45:34 - cmdstanpy - INFO - Chain [1] start processing
12:45:34 - cmdstanpy - INFO - Chain [1] done processing
12:45:35 - cmdstanpy - INFO - Chain [1] start processing
12:45:35 - cmdstanpy - INFO - Chain [1] done processing
12:45:35 - cmdstanpy - INFO - Chain [1] start processing
12:45:35 - cmdstanpy - INFO - Chain [1] done processing
12:45:35 - cmdstanpy - INFO - Chain [1] start processing
12:45:35 - cmdstanpy - INFO - Chain [1] done processing
12:45:35 - cmdstanpy - INFO - Chain [1] start processing
12:45:36 - cmdstanpy - INFO - Chain [1] done processing
12:45:36 - cmdstanpy - INFO - Chain [1] start processing
12:45:36 - cmdstanpy - INFO - Chain [1] done processing
12:45:36 - cmdstanpy - INFO - Chain [1] start processing
12:45:36 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,204.96,189.31,4.98,"[4232.15, 3536.13, 4060.78]","[4035.37, 3446.55, 3779.21]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,152.91,130.45,3.52,"[3498.45, 4019.08, 3591.47]","[3446.55, 3779.21, 3691.05]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,147.82,116.43,3.11,"[4012.2, 3585.42, 3585.52]","[3779.21, 3691.05, 3574.86]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,71.70,45.87,1.24,"[3567.27, 3566.16, 3946.29]","[3691.05, 3574.86, 3951.41]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,249.52,145.92,7.15,"[3570.18, 3952.35, 1597.32]","[3574.86, 3951.41, 2029.47]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,4135.42,2661.99,18.40,"[8586.5, 7137.9, 8704.9]","[15715.54, 6948.86, 9372.79]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,3437.12,2217.66,16.04,"[7659.53, 9341.09, 9827.45]","[6948.86, 9372.79, 15738.07]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,3580.13,2603.84,18.53,"[9357.79, 9832.47, 12457.42]","[9372.79, 15738.07, 10566.5]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,3681.23,3104.93,22.91,"[9845.21, 12469.43, 12960.98]","[15738.07, 10566.5, 11441.98]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,348.56,308.18,11.38
1,АКТЮБИНСКАЯ ОБЛАСТЬ,867.92,746.80,15.30
2,АЛМАТИНСКАЯ ОБЛАСТЬ,4097.33,3743.97,95.35
3,АТЫРАУСКАЯ ОБЛАСТЬ,305.92,269.17,18.16
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,3742.48,3410.64,111.41
5,ГАЛМАТЫ,13.42,11.62,795.82
6,ГАСТАНА,0.92,0.76,32.24
7,ГШЫМКЕНТ,416.82,344.15,69.97
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,690.90,566.08,13.11
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,234.01,201.49,4.83


In [16]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


11:52:17 - cmdstanpy - INFO - Chain [1] start processing
11:52:18 - cmdstanpy - INFO - Chain [1] done processing
11:52:18 - cmdstanpy - INFO - Chain [1] start processing
11:52:18 - cmdstanpy - INFO - Chain [1] done processing
11:52:18 - cmdstanpy - INFO - Chain [1] start processing
11:52:18 - cmdstanpy - INFO - Chain [1] done processing
11:52:18 - cmdstanpy - INFO - Chain [1] start processing
11:52:18 - cmdstanpy - INFO - Chain [1] done processing
11:52:19 - cmdstanpy - INFO - Chain [1] start processing
11:52:19 - cmdstanpy - INFO - Chain [1] done processing
11:52:19 - cmdstanpy - INFO - Chain [1] start processing
11:52:19 - cmdstanpy - INFO - Chain [1] done processing
11:52:19 - cmdstanpy - INFO - Chain [1] start processing
11:52:19 - cmdstanpy - INFO - Chain [1] done processing
11:52:20 - cmdstanpy - INFO - Chain [1] start processing
11:52:20 - cmdstanpy - INFO - Chain [1] done processing
11:52:20 - cmdstanpy - INFO - Chain [1] start processing
11:52:20 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,124.23,114.48,3.09,"[3902.77, 3284.17, 3730.77]","[4035.37, 3446.55, 3779.21]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,230.81,192.56,5.31,"[3279.52, 3728.15, 3331.45]","[3446.55, 3779.21, 3691.05]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,254.00,217.26,5.95,"[3739.88, 3337.62, 3315.85]","[3779.21, 3691.05, 3574.86]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,284.15,281.32,7.52,"[3363.24, 3344.64, 3665.48]","[3691.05, 3574.86, 3951.41]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,261.23,259.80,9.05,"[3352.93, 3679.26, 1744.16]","[3574.86, 3951.41, 2029.47]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,4772.16,3387.62,25.04,"[7631.6, 6545.58, 7697.15]","[15715.54, 6948.86, 9372.79]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,4068.24,2754.70,19.53,"[6851.98, 8143.19, 8800.45]","[6948.86, 9372.79, 15738.07]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,3984.88,2861.59,20.49,"[8283.51, 8960.47, 11284.4]","[9372.79, 15738.07, 10566.5]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,3811.60,2777.76,19.41,"[9274.82, 11678.71, 12199.79]","[15738.07, 10566.5, 11441.98]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,348.02,328.23,11.16
1,АКТЮБИНСКАЯ ОБЛАСТЬ,746.59,682.57,13.06
2,АЛМАТИНСКАЯ ОБЛАСТЬ,2458.44,2115.42,29.88
3,АТЫРАУСКАЯ ОБЛАСТЬ,345.15,316.49,17.81
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1381.16,1302.79,35.06
5,ГАЛМАТЫ,4.85,4.06,89.48
6,ГАСТАНА,0.65,0.58,26.30
7,ГШЫМКЕНТ,267.62,228.59,46.33
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,690.85,512.64,10.74
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,272.98,227.11,5.69


In [18]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW  MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ     4.77         5.22         11.16          HW
           АКТЮБИНСКАЯ ОБЛАСТЬ    10.54        12.77         13.06          HW
           АЛМАТИНСКАЯ ОБЛАСТЬ    12.93        11.63         29.88      SARIMA
            АТЫРАУСКАЯ ОБЛАСТЬ     8.87         8.61         17.81      SARIMA
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     9.46        10.50         35.06          HW
                       ГАЛМАТЫ   100.69        87.36         89.48      SARIMA
                       ГАСТАНА    15.60        26.88         26.30          HW
                      ГШЫМКЕНТ    43.10        28.85         46.33      SARIMA
            ЖАМБЫЛСКАЯ ОБЛАСТЬ    10.74         9.24         10.74      SARIMA
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     5.22         4.57          5.69      SARIMA
        КАРАГАНДИНСКАЯ ОБЛАСТЬ     6.67         8.47         22.68          HW
          КОСТАНАЙСКАЯ ОБЛАСТЬ    14.93        18.38